<div class='bar_title'></div>

*Enterprise AI*

# Final Project - Prediction of appendicitis

Group: Laura Buchmann (3060778), Tobias Stephan (2903743), Jonathan Szarka (3028005) <br>
Gunther Gust / Justus Ameling<br>
Chair of Enterprise AI

Summer Semester 2024

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true" style="width:20%; float:left;" />




# Instructions for reading our code

In this assignment, we develop a algorithm to predict an appendicitis diagnosis. <br>
<br>
- The starting point of our project is exploratory_data_analysis notebook, in which we identify the most important points in the data and draw conclusions for the pipeline structure.  <br>
- This structure consists of 1. feature_engineering_pipeline, 2. training_pipeline, 3. inference_pipeline and is subdivided into the respective steps that are noted in the individual pipelines. <br>
- We start with feature_engineering, in which we first load our data set (loading_data) and clean it up (data_cleaning).  <br>
- Based on this, in split_data the data set is split into train, test and validation data. <br>
- After that the measures for encoding, scaling and imputing the categorical and numerical data are defined in the create_preprocessing_pipeline. <br>
- In feature_preprocessor we transform the defined features. <br>
- In the last part of the feature engineering (label_encoding), label encoding of the target variable is applied. <br>
<br>
<br>
- The next part of our model is the training part.  <br> 
- In the step hp_tuning we perform hyperparameter tuning for either Random Forest or Decision Tree models using Optuna to maximize the F1 score. <br>
- The steps model_trainer_decision_tree and model_trainer_random_forest each train the models. <br>
- Next we evaluate the model in evaluate_model, which sets the foundation for the choice of the model. <br>
 <br>
  <br>
- Lastly there is the inference part where the model gets trained to make predictions on unseen data.  <br>
- The first step at this stage is the inference_data_loader where we load the data and drop the columns which we analyzed before as not significant for out prediction.  <br>
- The following inference_preprocessing transforms the inference data to prepare the inference prediction.  <br>
- In predictor the actual prediction takes place.  <br>
- In the end the prediction_loader loads the methods to predict.  <br>

In [11]:
!pip install -r requirements.txt
!pip install openpyxl
!pip install --upgrade zenml pydantic

  Using cached sqlmodel-0.0.8-py3-none-any.whl.metadata (9.7 kB)
Using cached sqlmodel-0.0.8-py3-none-any.whl (21 kB)
  Attempting uninstall: sqlmodel
    Found existing installation: sqlmodel 0.0.18
    Uninstalling sqlmodel-0.0.18:
      Successfully uninstalled sqlmodel-0.0.18
  Using cached zenml-0.61.0-py3-none-any.whl.metadata (20 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
  Using cached SQLAlchemy-2.0.31-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached sqlmodel-0.0.18-py3-none-any.whl.metadata (9.8 kB)
Using cached zenml-0.61.0-py3-none-any.whl (8.0 MB)
Using cached sqlmodel-0.0.18-py3-none-any.whl (26 kB)
Using cached pydantic-2.7.4-py3-none-any.whl (409 kB)
Using cached SQLAlchemy-2.0.31-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.41
    Un

In [12]:
!zenml init

Found existing ZenML repository at path '/workspaces/enterprise_ai'.
⠋ Initializing ZenML repository at /workspaces/enterprise_ai.
⠋ Initializing ZenML repository at /workspaces/enterprise_ai.



In [13]:
!zenml stack describe

Initializing the ZenML global configuration version to 0.61.0
Creating database tables
Creating default workspace 'default' ...
Creating default stack in workspace default...
Setting the global active workspace to 'default'.
Setting the global active stack to default.
The current repo active workspace is no longer available. Resetting the active workspace to 'default'.
The current repo active stack is no longer available. Resetting the active stack to default.
Reloading configuration file /workspaces/enterprise_ai/.zen/config.yaml
        Stack Configuration        
┏━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━┓
┃ COMPONENT_TYPE │ COMPONENT_NAME ┃
┠────────────────┼────────────────┨
┃ ORCHESTRATOR   │ default        ┃
┠────────────────┼────────────────┨
┃ ARTIFACT_STORE │ default        ┃
┗━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━┛
     'default' stack (ACTIVE)      
⠋ Describing the stack...
No labels are set for this stack.
⠋ Describing the stack...
Stack 'default' with id '035bc68c-6b57-4947-aebf-646d

In [14]:
!zenml integration install mlflow -y

⠴ Installing integrations.....


In [15]:
!zenml experiment-tracker register mlflow_experiment_tracker --flavor=mlflow

⠋ Registering experiment tracker 'mlflow_experiment_tracker'...
⠹ Registering experiment tracker 'mlflow_experiment_tracker'...
⠸ Registering experiment tracker 'mlflow_experiment_tracker'...
⠼ Registering experiment tracker 'mlflow_experiment_tracker'...
⠴ Registering experiment tracker 'mlflow_experiment_tracker'...
⠦ Registering experiment tracker 'mlflow_experiment_tracker'...
⠧ Registering experiment tracker 'mlflow_experiment_tracker'...
⠇ Registering experiment tracker 'mlflow_experiment_tracker'...
⠏ Registering experiment tracker 'mlflow_experiment_tracker'...
⠙ Registering experiment tracker 'mlflow_experiment_tracker'...
⠹ Registering experiment tracker 'mlflow_experiment_tracker'...
⠸ Registering experiment tracker 'mlflow_experiment_tracker'...
⠼ Registering experiment tracker 'mlflow_experiment_tracker'...
⠴ Registering experiment tracker 'mlflow_experiment_tracker'...
⠦ Registering experiment tracker 'mlflow_experiment_tracker'...
Successfully registered experiment_track

In [16]:
!zenml model-deployer register mlflow_deployer --flavor=mlflow

⠋ Registering model deployer 'mlflow_deployer'...
⠹ Registering model deployer 'mlflow_deployer'...
⠸ Registering model deployer 'mlflow_deployer'...
⠼ Registering model deployer 'mlflow_deployer'...
⠴ Registering model deployer 'mlflow_deployer'...
Successfully registered model_deployer `mlflow_deployer`.
⠦ Registering model deployer 'mlflow_deployer'...
You can display various ZenML entities including pipelines, runs, stacks and 
much more on the ZenML Dashboard. You can try it locally, by running `zenml up`,
or remotely, by deploying ZenML on the infrastructure of your choice.
⠦ Registering model deployer 'mlflow_deployer'...
⠦ Registering model deployer 'mlflow_deployer'...



In [17]:
!zenml stack register mlflow_stack -e mlflow_experiment_tracker -d mlflow_deployer -o default -a default --set

⠋ Registering stack 'mlflow_stack'...
⠙ Registering stack 'mlflow_stack'...
⠹ Registering stack 'mlflow_stack'...
⠸ Registering stack 'mlflow_stack'...
⠴ Registering stack 'mlflow_stack'...
⠦ Registering stack 'mlflow_stack'...
⠧ Registering stack 'mlflow_stack'...
⠇ Registering stack 'mlflow_stack'...
Stack 'mlflow_stack' successfully registered!
⠏ Registering stack 'mlflow_stack'...
               Stack Configuration                
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ COMPONENT_TYPE     │ COMPONENT_NAME            ┃
┠────────────────────┼───────────────────────────┨
┃ EXPERIMENT_TRACKER │ mlflow_experiment_tracker ┃
┠────────────────────┼───────────────────────────┨
┃ MODEL_DEPLOYER     │ mlflow_deployer           ┃
┠────────────────────┼───────────────────────────┨
┃ ORCHESTRATOR       │ default                   ┃
┠────────────────────┼───────────────────────────┨
┃ ARTIFACT_STORE     │ default                   ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


In [18]:
!zenml stack describe

               Stack Configuration                
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ COMPONENT_TYPE     │ COMPONENT_NAME            ┃
┠────────────────────┼───────────────────────────┨
┃ EXPERIMENT_TRACKER │ mlflow_experiment_tracker ┃
┠────────────────────┼───────────────────────────┨
┃ MODEL_DEPLOYER     │ mlflow_deployer           ┃
┠────────────────────┼───────────────────────────┨
┃ ORCHESTRATOR       │ default                   ┃
┠────────────────────┼───────────────────────────┨
┃ ARTIFACT_STORE     │ default                   ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
          'mlflow_stack' stack (ACTIVE)           
⠋ Describing the stack...
           Labels           
┏━━━━━━━━━━━━━━━━━━┯━━━━━━━┓
┃ LABEL            │ VALUE ┃
┠──────────────────┼───────┨
┃ zenml:full_stack │ True  ┃
┗━━━━━━━━━━━━━━━━━━┷━━━━━━━┛
⠋ Describing the stack...
Stack 'mlflow_stack' with id 'f6953980-9ef9-4674-b99b-0ed5f1481347' is owned by 
user default.
⠋ Describing the

In [19]:
!python run.py

Initiating a new run for the pipeline: feature_engineering_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: mlflow_stack
  experiment_tracker: mlflow_experiment_tracker
  model_deployer: mlflow_deployer
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Caching disabled explicitly for loading_data.
Step loading_data has started.
Step loading_data has finished in 1.902s.
Step loading_data completed successfully.
Caching disabled explicitly for data_cleaning.
Step data_cleaning has started.
Step data_cleaning has finished in 0.651s.
Step data_cleaning completed successfully.
Step create_preprocessing_pipeline has started.
Step create_preprocessing_pipeline has finished in 0.242s.
Step create_preprocessing_pipeline completed successfully.
Step split_data has started.
Failed to extract metadata for output artifact 'y_train': Could 

In [20]:
from zenml.client import Client
pipeline_run = Client().get_pipeline("training_pipeline").last_run
step = pipeline_run.steps["model_trainer_random_forest"]
experiment_tracker_url = step.run_metadata["experiment_tracker_url"].value
experiment_tracker_url

'file:/home/codespace/.config/zenml/local_stores/f8d31cfe-bccf-4240-b139-aeea3acf09a2/mlruns'

In [21]:
!mlflow ui --backend-store-uri 'file:/home/codespace/.config/zenml/local_stores/f8d31cfe-bccf-4240-b139-aeea3acf09a2/mlruns'

/usr/local/python/3.10.13/lib/python3.10/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2024-07-15 08:50:03 +0000] [9055] [INFO] Starting gunicorn 21.2.0
[2024-07-15 08:50:03 +0000] [9055] [INFO] Listening at: http://127.0.0.1:5000 (9055)
[2024-07-15 08:50:03 +0000] [9055] [INFO] Using worker: sync
[2024-07-15 08:50:03 +0000] [9056] [INFO] Booting worker with pid: 9056
[2024-07-15 08:50:03 +0000] [9057] [INFO] Booting worker with pid: 9057
[2024-07-15 08:50:03 +0000] [9058] [INFO] Booting worker with pid: 9058
[2024-07-15 08:50:03 +0000] [9059] [INFO] Booting worker with pid: 9059
^C
[2024-07-15 08:50:22 +0000] [9055] [INFO] Handling signal: int
[2024-07-15 08:50:22 +0000] [9058] [INFO] Worker exiting (pid: 9058)
[2024-07-15 08:50:22 +0000] [9056] [INFO] Worker exiting (pid: 9056)
[2024-07-15 08:50:22 +0000] [9057] [INFO] Worker exiting (p

In [ ]:
# ZenML Dashboard to see visualization of current pipelines
!zenml up

# To open, click on the link below. If you do not have a ZenML account, enter "default" as the user name and leave the password blank.

Deploying a local ZenML server with name 'local'.
Calling start method...
⠋ Starting service 'LocalZenServer[73f66782-44d5-404a-952a-e0b60bc9a6e5] (type: 
zen_server, flavor: local)'.
⠙ Starting service 'LocalZenServer[73f66782-44d5-404a-952a-e0b60bc9a6e5] (type: 
zen_server, flavor: local)'.
⠹ Starting service 'LocalZenServer[73f66782-44d5-404a-952a-e0b60bc9a6e5] (type: 
zen_server, flavor: local)'.
⠸ Starting service 'LocalZenServer[73f66782-44d5-404a-952a-e0b60bc9a6e5] (type: 
zen_server, flavor: local)'.
⠼ Starting service 'LocalZenServer[73f66782-44d5-404a-952a-e0b60bc9a6e5] (type: 
zen_server, flavor: local)'.
⠴ Starting service 'LocalZenServer[73f66782-44d5-404a-952a-e0b60bc9a6e5] (type: 
zen_server, flavor: local)'.
⠦ Starting service 'LocalZenServer[73f66782-44d5-404a-952a-e0b60bc9a6e5] (type: 
zen_server, flavor: local)'.
⠧ Starting service 'LocalZenServer[73f66782-44d5-404a-952a-e0b60bc9a6e5] (type: 
zen_server, flavor: local)'.
⠇ Starting service 'LocalZenServer[73f66782-44

Feature_engineering_pipeline

<img src="./images/feature_engineering.PNG" style="width:100%;" />

Training_pipeline

<img src="./images/training.PNG" style="width:90%;" />

Inference_pipeline

<img src="./images/inference.PNG" style="width:40%;" />

In [36]:
import pandas as pd
predictions = pd.read_csv("./predictions.csv")
inference = pd.read_excel("./data/inference.xlsx")
columns_to_drop = ['Diagnosis', 'Length_of_Stay' , 'Management' , 'Severity' , 'Diagnosis_Presumptive' , 'Segmented_Neutrophils' , 'Appendix_Wall_Layers' , 'Target_Sign' , 'Appendicolith' , 'Perfusion' , 'Perforation' , 'Surrounding_Tissue_Reaction' , 'Appendicular_Abscess' , 'Abscess_Location' , 'Pathological_Lymph_Nodes' , 'Lymph_Nodes_Location' , 'Bowel_Wall_Thickening' , 'Conglomerate_of_Bowel_Loops' , 'Ileus' , 'Coprostasis' , 'Meteorism' , 'Enteritis' , 'Gynecological_Findings']
inference = inference.drop(columns=columns_to_drop)
predictions['prediction'] = predictions['prediction'].map({1: "no appendicitis", 0: "appendicitis"})
df_combined = pd.concat([predictions, inference], axis=1)
df_combined


,prediction,Age,BMI,Sex,Height,Weight,Alvarado_Score,Paedriatic_Appendicitis_Score,Appendix_on_US,Appendix_Diameter,...,WBC_in_Urine,CRP,Dysuria,Stool,Peritonitis,Psoas_Sign,Ipsilateral_Rebound_Tenderness,US_Performed,US_Number,Free_Fluids
0,no appendicitis,15.660000,20.100000,female,164.0,54.00,3,3,yes,6.0,...,+,0,no,normal,local,yes,no,yes,988,no
1,no appendicitis,8.560000,16.300000,female,138.0,30.50,4,3,yes,4.0,...,no,0,no,normal,no,yes,no,yes,603,yes
2,no appendicitis,17.489391,17.722681,female,165.0,48.25,2,2,no,NaN,...,+,0,no,normal,no,no,no,yes,320,no
3,no appendicitis,14.614648,11.034405,male,188.0,39.00,6,5,yes,3.5,...,no,42,no,normal,no,no,no,yes,437,no
4,no appendicitis,14.099932,18.818924,male,163.0,50.00,2,2,no,NaN,...,no,1,yes,normal,no,no,no,yes,293,yes
5,no appendicitis,10.995209,22.456083,female,156.5,55.00,4,3,yes,2.9,...,no,0,no,diarrhea,no,no,yes,yes,228,NaN
6,appendicitis,11.420000,14.540000,male,140.0,28.50,8,6,yes,8.0,...,+,0,NaN,diarrhea,local,no,NaN,yes,536,yes
7,appendicitis,11.050000,15.320000,female,131.5,26.50,6,7,no,NaN,...,+++,171,no,constipation,local,no,no,yes,481,no
8,appendicitis,9.275838,16.251154,male,142.5,33.00,5,4,yes,7.5,...,no,1,no,normal,local,no,no,yes,132,no
9,appendicitis,13.486653,18.426534,female,158.0,46.00,2,2,yes,10.9,...,no,5,NaN,normal,local,no,no,yes,108,no
